#Range Program
Ghost Okaamii
#
#
This program is for measuring ranges accross different assets and time frames. 

In [16]:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def calculate_pivots(df):
    """Calculate floor pivot points"""
    pivot = (df['high'].shift(1) + df['low'].shift(1) + df['close'].shift(1)) / 3
    r1 = (2 * pivot) - df['low'].shift(1)
    r2 = pivot + (df['high'].shift(1) - df['low'].shift(1))
    r3 = df['high'].shift(1) + 2*(pivot - df['low'].shift(1))
    s1 = (2 * pivot) - df['high'].shift(1)
    s2 = pivot - (df['high'].shift(1) - df['low'].shift(1))
    s3 = df['low'].shift(1) - 2*(df['high'].shift(1) - pivot)
    
    return pivot, r1, r2, r3, s1, s2, s3

def calculate_ranges(df, periods):
    """Calculate various ranges over different periods"""
    for period in periods:
        # High to Low range
        df[f'{period}dayrange'] = df['high'].rolling(window=period).max() - df['low'].rolling(window=period).min()
        
        # High to Close range
        df[f'{period}day_hi2close'] = df['high'].rolling(window=period).max() - df['close']
        
        # Low to Close range
        df[f'{period}day_lo2close'] = df['close'] - df['low'].rolling(window=period).min()

def get_nq_futures_data(period="2y", interval="1d"):
    """
    Download NQ futures data using yfinance and include pivots, moving averages, and ranges
    """
    try:
        # NQ futures ticker symbol
        ticker = "NQ=F"
        
        # Create ticker object and download with more history
        nq = yf.Ticker(ticker)
        
        # Download data with progress info
        print("Downloading data...")
        df = nq.history(period=period, interval=interval)
        print(f"Downloaded {len(df)} rows of data")
        
        if df.empty:
            print("Error: No data received from yfinance")
            return None
        
        # Reset index to make Date a column
        df.reset_index(inplace=True)
        
        # Clean up column names
        df.columns = df.columns.str.lower()
        
        # Add settlement price
        df['settlement'] = df['close']
        
        # Calculate and add pivot points
        df['pivot'], df['r1'], df['r2'], df['r3'], df['s1'], df['s2'], df['s3'] = calculate_pivots(df)
        
        # Calculate moving averages
        ma_periods = [5, 10, 20, 50, 100, 200]
        for ma in ma_periods:
            df[f'ma{ma}'] = df['close'].rolling(window=ma).mean()
        
        # Calculate ranges for specified periods
        range_periods = [2, 3, 5, 10, 20, 30, 50, 100, 200, 300]
        calculate_ranges(df, range_periods)
        
        # Create list of columns in desired order
        base_cols = ['date', 'open', 'high', 'low', 'close', 'settlement']
        pivot_cols = ['pivot', 'r1', 'r2', 'r3', 's1', 's2', 's3']
        ma_cols = [f'ma{ma}' for ma in ma_periods]
        range_cols = []
        for period in range_periods:
            range_cols.extend([
                f'{period}dayrange',
                f'{period}day_hi2close',
                f'{period}day_lo2close'
            ])
        
        # Combine all columns in order
        cols = base_cols + pivot_cols + ma_cols + range_cols + ['volume']
        
        # Make sure all required columns exist
        existing_cols = [col for col in cols if col in df.columns]
        df = df[existing_cols]
        
        return df
    
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

# Get daily data
df = get_nq_futures_data(period="2y")

if df is not None and not df.empty:
    print("\nFirst few rows of the data:")
    display(df.head())
    
    print("\nDataFrame info:")
    display(df.info())
else:
    print("Failed to retrieve data. Please check your internet connection and try again.")


Downloaded 505 rows of data

First few rows of the data:


,date,open,high,low,close,settlement,pivot,r1,r2,r3,...,100dayrange,100day_hi2close,100day_lo2close,200dayrange,200day_hi2close,200day_lo2close,300dayrange,300day_hi2close,300day_lo2close,volume
0,2022-12-19 00:00:00-05:00,11328.75,11411.00,11131.75,11193.25,11193.25,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,562529
1,2022-12-20 00:00:00-05:00,11182.50,11247.00,11043.50,11175.75,11175.75,11245.333333,11358.916667,11524.583333,11638.166667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,625090
2,2022-12-21 00:00:00-05:00,11206.50,11389.75,11181.00,11334.50,11334.50,11155.416667,11267.333333,11358.916667,11470.833333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,631322
3,2022-12-22 00:00:00-05:00,11357.00,11384.75,10870.50,11054.25,11054.25,11301.750000,11422.500000,11510.500000,11631.250000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,714500
4,2022-12-23 00:00:00-05:00,11061.00,11143.00,10916.25,11075.25,11075.25,11103.166667,11335.833333,11617.416667,11850.083333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,534525



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 50 columns):
 #   Column           Non-Null Count  Dtype                           
---  ------           --------------  -----                           
 0   date             505 non-null    datetime64[ns, America/New_York]
 1   open             505 non-null    float64                         
 2   high             505 non-null    float64                         
 3   low              505 non-null    float64                         
 4   close            505 non-null    float64                         
 5   settlement       505 non-null    float64                         
 6   pivot            504 non-null    float64                         
 7   r1               504 non-null    float64                         
 8   r2               504 non-null    float64                         
 9   r3               504 non-null    float64                         
 10  s1               504 

None